In [7]:
import sys

from cellpose import models, core
from tifffile import imread
from tifffile import imwrite
from glob import glob
import numpy as np
import time
from glob import glob
import os

load_path = "G:/Data/IBIN_Nina/workspace/nina_cellpose/IBIN_Nina_code/example_volume/drug_plate3_field002_t0_exc561_filter605.tif"
# load_path = "G:/Data/IBIN_Nina/workspace/trained_masks_new_model/l19_t9/L19_t9.tif"


stack = imread(load_path)

cellpose2_model_path = "G:/Data/IBIN_Nina/workspace/nina_cellpose/training_data/with_reslice/model/models/cellpose_residual_on_style_on_concatenation_off__2022_10_28_14_29_47.213939_epoch_151"
cellpose3_model_path = "G:/Data/IBIN_Nina/workspace/nina_cellpose/IBIN_Nina_code/models/nina_model"

model_path = cellpose3_model_path

savepath = "G:/Data/IBIN_Nina/workspace/nina_cellpose/IBIN_Nina_code/example_volume/segmentation"


diam_mean = 13

print("segmenting %s" % load_path)

## remove bg from data
stack = np.asarray(stack, dtype=np.int32)
stack -= 100  #subtract zero signed int
negative_mask = stack > 0
stack = stack*negative_mask  # remove negative values
stack = np.asarray(stack, dtype=np.uint16)  # convert back to unsigned

use_default_model = False

model = models.CellposeModel(pretrained_model = model_path,
                                diam_mean = diam_mean,
                                model_type=None,
                                gpu=True,
                                # torch = True,
                                # net_avg = True
                            )

default_model = models.CellposeModel(diam_mean = diam_mean,
                                 model_type='nuclei',
                                 gpu=True,
                                 # torch = True,
                                 # net_avg = True
                                )

if use_default_model:
    model = default_model


# model = models.CellposeModel(model_type=model_path,
#                        diam_mean = diam_mean,
#                         gpu=True,
#                    )
start_time = time.time()

output = model.eval(stack, channels=[0,0], do_3D=True, diameter=diam_mean, batch_size=1)
masks = output[0]

segment_time = time.time() - start_time
print("Time taken to segment:", segment_time, "s")

## save
print("saving:", savepath)
if not os.path.exists(savepath):
    os.mkdir(savepath)
imwrite(os.path.join(savepath, "cellpose_segmentation_drug_newmodel.tif"), np.asarray(masks, dtype=np.uint16))

segmenting G:/Data/IBIN_Nina/workspace/nina_cellpose/IBIN_Nina_code/example_volume/drug_plate3_field002_t0_exc561_filter605.tif
Time taken to segment: 371.6056299209595 s
saving: G:/Data/IBIN_Nina/workspace/nina_cellpose/IBIN_Nina_code/example_volume/segmentation


In [33]:
model2 = models.CellposeModel(pretrained_model = cellpose2_model_path,
                                diam_mean = diam_mean,
                                model_type=None,
                                gpu=True,
                            )
model3 = models.CellposeModel(pretrained_model = cellpose3_model_path,
                                diam_mean = diam_mean,
                                model_type=None,
                                gpu=True,
                            )

print(model2.gpu)
print(model2.device)
print(model2.net_type)
print(model2.nbase)
print(model2.diam_mean)

print(model3.gpu)
print(model3.device)
print(model3.net_type)
print(model3.nbase)
print(model3.diam_mean)

# Model I trained in Cellpose 3 fixes diameter to be 30, i.e., dataset is rescaled in training to 30, while old model is
# trained for a diameter of 13 which matches the acutal input/inference data. I think the reason this Cellpose 3 model runs much
# slower is because of this: the input data is scaled from 13 to 30 during inference (and scaled back?). 
# 
# For the nuclei model (which I thought I was using) as a pretrained base, the diameter should be fixed to 17 instead, so the
# fact it says 30 here makes it seems like I've made a mistake and it's training off the wrong model? I may just retrain from
# scratch and that may be necessary, including Cellpose dataset or not 

True
cuda:0
cellpose
[2, 32, 64, 128, 256]
13.0
True
cuda:0
cellpose
[2, 32, 64, 128, 256]
30.0


In [22]:
model2 = models.CellposeModel(pretrained_model = cellpose2_model_path,
                                diam_mean = diam_mean,
                                model_type=None,
                                gpu=True,
                                # torch = True,
                                # net_avg = True
                            )

model3 = models.CellposeModel(pretrained_model = cellpose3_model_path,
                                diam_mean = diam_mean,
                                model_type=None,
                                gpu=True,
                                # torch = True,
                                # net_avg = True
                            )

print("=============Model from cellpose 2=============")
print(model2.pretrained_model)
#print(model2.net.named_parameters)
print(model2.net.parameters)


print("=============Model from cellpose 3=============")
print(model3.pretrained_model)
print(model3.net.parameters)
model.gpu



=============Model from cellpose 2=============
G:/Data/IBIN_Nina/workspace/nina_cellpose/training_data/with_reslice/model/models/cellpose_residual_on_style_on_concatenation_off__2022_10_28_14_29_47.213939_epoch_151
<bound method Module.parameters of CPnet(
  (downsample): downsample(
    (down): Sequential(
      (res_down_0): resdown(
        (conv): Sequential(
          (conv_0): Sequential(
            (0): BatchNorm2d(2, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (conv_1): Sequential(
            (0): BatchNorm2d(32, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (conv_2): Sequential(
            (0): BatchNorm2d(32, eps=1e-05, momentum=0.05, affine=True, track_run

In [6]:
os.mkdir(savepath)
imwrite(os.path.join(savepath, "cellpose_segmentation.tif"), np.asarray(masks, dtype=np.uint16))